# General Idea 
Creating a machine, that will analyse the occurences of different source patterns in a lower (finer) time frame to a higher time frame i.e. daily -> weekly. 
- Is it possible to generate some insights, which patterns lead more probably to certain result patterns (target prices)?
![rule analyzer](rule_analyzer.png)

In [1]:
#%pip install numpy --upgrade
#%pip install alpha_vantage
#%pip install pandas_ta

In [2]:
import pandas as pd
import pandas_ta as ta
from alpha_vantage.timeseries import TimeSeries

## 1. Step: Get the data

In [ ]:
stock_ticker = 'MSFT'

In [4]:
ts = TimeSeries(key='X6SGFO2II4530GDY', output_format='pandas')
# Get json object with the intraday data and another with  the call's metadata
daily_data, daily_meta_data = ts.get_daily_adjusted(stock_ticker, )
weekly_data, weekly_meta_data = ts.get_weekly_adjusted(stock_ticker)

In [5]:
daily_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend', 'split_coeff']
weekly_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'dividend']
daily_data = daily_data[['open', 'high', 'low', 'close', 'adjusted_close']]
weekly_data = weekly_data[['open', 'high', 'low', 'close', 'adjusted_close']]
daily_data = daily_data.sort_index(ascending=True)
weekly_data = weekly_data.sort_index(ascending=True)

## 2. Step: Get technical data

In [6]:
CustomStrategy = ta.Strategy(
    name="Hlouschek",
    description="MA 9, MA 20, BB2, BB3, %B2, ADX, DI+, DI-, Parabolic SAR",
    ta=[
        {"kind": "sma", "length": 9},
        {"kind": "sma", "length": 20},
        {"kind": "bbands", "length": 20, "std": 2}, #includes percent_b
        {"kind": "bbands", "length": 20, "std": 3}, #includes percent_b
        {"kind": "adx"}, #default parameters are ok
        {"kind": "psar"}, #default parameters are ok
        
        # eventuell später relevant für options for winners indicator
        #{"kind": "macd", "fast": 8, "slow": 21},
        #{"kind": "sma", "close": "volume", "length": 20, "prefix": "VOLUME"},
    ]
)
# Run "Custom Strategy"
daily_data.ta.strategy(CustomStrategy)
weekly_data.ta.strategy(CustomStrategy)

In [7]:
# daily data munging
daily_data['psar'] = daily_data['PSARl_0.02_0.2'].combine_first(daily_data['PSARs_0.02_0.2'])
daily_data = daily_data[['open', 'high', 'low', 'close', 'adjusted_close', 'SMA_9', 'SMA_20',
       'BBL_20_2.0', 'BBU_20_2.0', 'BBP_20_2.0', 'BBL_20_3.0', 'BBU_20_3.0', 'BBP_20_3.0',
       'ADX_14', 'DMP_14', 'DMN_14', 'psar']]
daily_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'sma_9', 'sma_20',
       'bbl_2', 'bbu_2', 'bbp_2', 'bbl_3', 'bbu_3', 'bbp_3', 'adx_14', 'dmp_14', 'dmn_14', 'psar']
for c in daily_data.columns:
    daily_data[c + '_t_minus_1'] = daily_data[c].shift(1)

In [8]:
# weekly data munging
weekly_data['psar'] = weekly_data['PSARl_0.02_0.2'].combine_first(weekly_data['PSARs_0.02_0.2'])
weekly_data = weekly_data[['open', 'high', 'low', 'close', 'adjusted_close', 'SMA_9', 'SMA_20',
       'BBL_20_2.0', 'BBU_20_2.0', 'BBP_20_2.0', 'BBL_20_3.0', 'BBU_20_3.0', 'BBP_20_3.0',
       'ADX_14', 'DMP_14', 'DMN_14', 'psar']]
weekly_data.columns = ['open', 'high', 'low', 'close', 'adjusted_close', 'sma_9', 'sma_20',
       'bbl_2', 'bbu_2', 'bbp_2', 'bbl_3', 'bbu_3', 'bbp_3', 'adx_14', 'dmp_14', 'dmn_14', 'psar']
for c in weekly_data.columns:
    weekly_data[c + '_t_minus_1'] = weekly_data[c].shift(1)

In [9]:
# checkpoint
dd = daily_data
wd = weekly_data

## 3. Step: Create the rules for generating events

In [10]:
dd.columns

In [11]:
def dummy_rule(df):
    if (df['close'] > 1):
        return ' -> dummy_string'
    else:
        return ''

#########################
### BULLISH CROSSINGS ###
#########################
    
def rule_adx_bull_crossing(df):
    if (df['dmp_14'] > df['dmn_14']) and (df['adx_14_t_minus_1'] < df['dmn_14_t_minus_1']) and (df['adx_14'] > df['dmn_14']):
        return ' -> adx_bull_crossing'
    else:
        return ''
    
def rule_price_bull_crossing_9ma(df):
    if (df['close_t_minus_1'] < df['sma_9_t_minus_1'] and df['close'] > df['sma_9']):
        return ' -> price_bull_crossing_9ma'
    else:
        return ''

def rule_price_bull_crossing_20ma(df):
    if (df['close_t_minus_1'] < df['sma_20_t_minus_1'] and df['close'] > df['sma_20']):
        return ' -> price_bull_crossing_20ma'
    else:
        return ''

def rule_price_bull_crossing_psar(df):
    if (df['close_t_minus_1'] < df['psar_t_minus_1'] and df['close'] > df['psar']):
        return ' -> price_bull_crossing_psar'
    else:
        return ''

def rule_price_bull_crossing_l_bb2(df):
    if (df['close_t_minus_1'] < df['bbl_2_t_minus_1'] and df['close'] > df['bbl_2']):
        return ' -> price_bull_crossing_lower_bb2'
    else:
        return ''

def rule_price_bull_crossing_l_bb3(df):
    if (df['close_t_minus_1'] < df['bbl_3_t_minus_1'] and df['close'] > df['bbl_3']):
        return ' -> price_bull_crossing_lower_bb3'
    else:
        return ''
    
def rule_price_bull_crossing_u_bb2(df):
    if (df['close_t_minus_1'] < df['bbu_2_t_minus_1'] and df['close'] > df['bbu_2']):
        return ' -> price_bull_crossing_upper_bb2'
    else:
        return ''

def rule_price_bull_crossing_u_bb3(df):
    if (df['close_t_minus_1'] < df['bbu_3_t_minus_1'] and df['close'] > df['bbu_3']):
        return ' -> price_bull_crossing_upper_bb3'
    else:
        return ''

def rule_bullish_crossing_percb_over_0(df):
    if (df['bbp_2_t_minus_1'] < 0 and df['bbp_2'] > 0):
        return ' -> percb_bull_crossing_over_0'
    else:
        return ''

def rule_bullish_crossing_percb_over_1(df):
    if (df['bbp_2_t_minus_1'] < 1 and df['bbp_2'] > 1):
        return ' -> percb_bull_crossing_over_1'
    else:
        return ''

def rule_dmp_bullish_crossing_over_dmn(df):
    if (df['dmp_14_t_minus_1'] < df['dmn_14_t_minus_1'] and df['dmp_14'] > df['dmn_14']):
        return ' -> di+_bull_crossing_di-'
    else:
        return ''

def rule_dmn_bullish_crossing_over_dmp(df):
    if (df['dmn_14_t_minus_1'] < df['dmp_14_t_minus_1'] and df['dmn_14'] > df['dmp_14']):
        return ' -> di-_bull_crossing_di+'
    else:
        return ''
    
#########################
### BAERISH CROSSINGS ###
#########################

def rule_adx_bear_crossing(df):
    if (df['dmn_14'] > df['dmp_14']) and (df['adx_14_t_minus_1'] < df['dmn_14_t_minus_1']) and (df['adx_14'] > df['dmn_14']):
        return ' -> adx_bear_crossing'
    else:
        return ''

def rule_price_bear_crossing_9ma(df):
    if (df['close_t_minus_1'] > df['sma_9_t_minus_1'] and df['close'] < df['sma_9']):
        return ' -> price_bear_crossing_9ma'
    else:
        return ''

def rule_price_bear_crossing_20ma(df):
    if (df['close_t_minus_1'] > df['sma_20_t_minus_1'] and df['close'] < df['sma_20']):
        return ' -> price_bear_crossing_20ma'
    else:
        return ''

def rule_price_bear_crossing_psar(df):
    if (df['close_t_minus_1'] > df['psar_t_minus_1'] and df['close'] < df['psar']):
        return ' -> price_bear_crossing_psar'
    else:
        return ''

def rule_price_bear_crossing_l_bb2(df):
    if (df['close_t_minus_1'] > df['bbl_2_t_minus_1'] and df['close'] < df['bbl_2']):
        return ' -> price_bear_crossing_lower_bb2'
    else:
        return ''

def rule_price_bear_crossing_l_bb3(df):
    if (df['close_t_minus_1'] > df['bbl_3_t_minus_1'] and df['close'] < df['bbl_3']):
        return ' -> price_bear_crossing_lower_bb3'
    else:
        return ''
    
def rule_price_bear_crossing_u_bb2(df):
    if (df['close_t_minus_1'] > df['bbu_2_t_minus_1'] and df['close'] < df['bbu_2']):
        return ' -> price_bear_crossing_upper_bb2'
    else:
        return ''

def rule_price_bear_crossing_u_bb3(df):
    if (df['close_t_minus_1'] > df['bbu_3_t_minus_1'] and df['close'] < df['bbu_3']):
        return ' -> price_bear_crossing_upper_bb3'
    else:
        return ''

def rule_bearish_crossing_percb_over_0(df):
    if (df['bbp_2_t_minus_1'] > 0 and df['bbp_2'] < 0):
        return ' -> percb_bear_crossing_over_0'
    else:
        return ''

def rule_bearish_crossing_percb_over_1(df):
    if (df['bbp_2_t_minus_1'] > 1 and df['bbp_2'] < 1):
        return ' -> percb_bear_crossing_over_1'
    else:
        return ''

def rule_dmp_bearish_crossing_over_dmn(df):
    if (df['dmp_14_t_minus_1'] > df['dmn_14_t_minus_1'] and df['dmp_14'] < df['dmn_14']):
        return ' -> di+_bear_crossing_di-'
    else:
        return ''

def rule_dmn_bearish_crossing_over_dmp(df):
    if (df['dmn_14_t_minus_1'] > df['dmp_14_t_minus_1'] and df['dmn_14'] < df['dmp_14']):
        return ' -> di-_bear_crossing_di+'
    else:
        return ''

In [12]:
daily_rules = [
    #dummy_rule,
    rule_adx_bull_crossing,
    rule_price_bull_crossing_9ma,
    rule_price_bull_crossing_20ma,
    rule_price_bull_crossing_psar,
    rule_price_bull_crossing_l_bb2,
    rule_price_bull_crossing_l_bb3,
    rule_price_bull_crossing_u_bb2,
    rule_price_bull_crossing_u_bb3,
    rule_bullish_crossing_percb_over_0,
    rule_bullish_crossing_percb_over_1,
    rule_dmp_bullish_crossing_over_dmn,
    rule_dmn_bullish_crossing_over_dmp,
    rule_adx_bear_crossing,
    rule_price_bear_crossing_9ma,
    rule_price_bear_crossing_20ma,
    rule_price_bear_crossing_psar,
    rule_price_bear_crossing_l_bb2,
    rule_price_bear_crossing_l_bb3,
    rule_price_bear_crossing_u_bb2,
    rule_price_bear_crossing_u_bb3,
    rule_bearish_crossing_percb_over_0,
    rule_bearish_crossing_percb_over_1,
    rule_dmp_bearish_crossing_over_dmn,
    rule_dmn_bearish_crossing_over_dmp,
    
    ]

weekly_rules = [
    #dummy_rule,
    rule_adx_bull_crossing,
    rule_price_bull_crossing_9ma,
    rule_price_bull_crossing_20ma,
    rule_price_bull_crossing_psar,
    rule_price_bull_crossing_l_bb2,
    rule_price_bull_crossing_l_bb3,
    rule_price_bull_crossing_u_bb2,
    rule_price_bull_crossing_u_bb3,
    rule_bullish_crossing_percb_over_0,
    rule_bullish_crossing_percb_over_1,
    rule_dmp_bullish_crossing_over_dmn,
    rule_dmn_bullish_crossing_over_dmp,
    rule_adx_bear_crossing,
    rule_price_bear_crossing_9ma,
    rule_price_bear_crossing_20ma,
    rule_price_bear_crossing_psar,
    rule_price_bear_crossing_l_bb2,
    rule_price_bear_crossing_l_bb3,
    rule_price_bear_crossing_u_bb2,
    rule_price_bear_crossing_u_bb3,
    rule_bearish_crossing_percb_over_0,
    rule_bearish_crossing_percb_over_1,
    rule_dmp_bearish_crossing_over_dmn,
    rule_dmn_bearish_crossing_over_dmp,
]

In [13]:
def rule_execution(df, rules, event_class):
    liste = []
    for index, row in df.iterrows():
        resp = ''
        for rule in rules:
            resp += rule(row)
        liste.append((index, resp))
    res = pd.DataFrame(liste)
    res.columns = ['date', event_class]
    res = res.set_index('date')
    return res

In [14]:
daily_rule_results = rule_execution(dd, daily_rules, 'daily_event')
weekly_rule_results = rule_execution(wd, weekly_rules, 'weekly_event')

# My tests

In [15]:
daily_rule_results.loc[(daily_rule_results.index.month == 6)]

In [16]:
weekly_rule_results.loc[(weekly_rule_results.index.year==2021) & (weekly_rule_results.index.month==6)]

## 4. Step: Join on time

In [22]:
df_merged = pd.concat([dd,daily_rule_results, weekly_rule_results], axis=1)
#reduced = df_merged.loc[(df_merged.index.month==6) & (df_merged.index.year==2021) & (df_merged.daily_event != '' ),['daily_event', 'weekly_event']]

## 5. Step: Create Event String for Preceeding event

In [23]:
reduced = df_merged.loc[(df_merged.daily_event != '' ),['daily_event', 'weekly_event']]
reduced['weekly_event_preceding_after_this_daily_event'] = reduced['weekly_event'].shift(-1)
reduced_not_null = reduced[reduced.weekly_event_preceding_after_this_daily_event.notnull()]
reduced_not_null = reduced_not_null.loc[(reduced_not_null.weekly_event_preceding_after_this_daily_event != ''),['daily_event', 'weekly_event_preceding_after_this_daily_event']]
final_source = reduced_not_null[reduced_not_null.daily_event.notnull()]
final_source

,daily_event,weekly_event_preceding_after_this_daily_event
date,,
2021-03-25,-> price_bear_crossing_9ma,-> price_bull_crossing_9ma
2021-04-22,-> price_bear_crossing_9ma -> price_bear_cros...,-> price_bear_crossing_upper_bb2 -> percb_bea...
2021-05-13,-> price_bull_crossing_lower_bb2 -> percb_bul...,-> price_bear_crossing_9ma
2021-06-10,-> price_bull_crossing_upper_bb2 -> percb_bul...,-> price_bull_crossing_9ma
2021-06-22,-> adx_bull_crossing,-> price_bull_crossing_upper_bb2 -> percb_bul...
2021-07-22,-> price_bull_crossing_psar,-> price_bull_crossing_upper_bb2 -> percb_bul...
2021-07-23,-> price_bull_crossing_upper_bb2 -> percb_bul...,-> price_bear_crossing_upper_bb2 -> percb_bea...


## 6. Step: Calc and sort occurrences

In [19]:
# select distinct daily_event, weekly_event from XY
# select daily_event, count(*) as all_events_after_a from XY group by daily_event
# select daily_event, weekly_event from XY group by daily_event, weekly_event